# Segmenting and Clustering Neighborhoods in Toronto

Import all the libraries required.

In [76]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize

import json

import requests

import csv

print("LIBRARIES IMPORTED")

LIBRARIES IMPORTED


Import BeautifulSoup and the additional libraries required for it.

In [77]:
!python3.6 -m ensurepip --default-pip
!python3.6 -m pip install beautifulsoup4
!python3.6 -m pip install lxml
!python3.6 -m pip install html5lib
from bs4 import BeautifulSoup

Get the text at the url given. For this use requests.

In [78]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

Now that you have the source text,extract the table and put it into a CSV file. And then, extract the CSV into a dataframe using pd.read_csv.

In [79]:
soup = BeautifulSoup(source,'lxml')


csv_file = open("toronto_scrape.csv",'w')
csv_writer = csv.writer(csv_file)


toronto_table = soup.find('table', class_= 'wikitable sortable')

for tr in toronto_table.tbody.find_all('tr'):
     csv_writer.writerow(tr.text.split("\n"))

csv_file.close()

csv_file = open("toronto_scrape.csv",'r')
df = pd.read_csv(csv_file)

csv_file.close()

Let's check how the dataframe looks..

In [80]:
df.head(5)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Unnamed: 4
0,NaN,M1A,Not assigned,Not assigned,NaN
1,NaN,M2A,Not assigned,Not assigned,NaN
2,NaN,M3A,North York,Parkwoods,NaN
3,NaN,M4A,North York,Victoria Village,NaN
4,NaN,M5A,Downtown Toronto,Harbourfront,NaN


Remove the unnecesary columns from the dataframe.

In [81]:
df.drop(['Unnamed: 0','Unnamed: 4'], axis=1, inplace = True)
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [82]:
df1 = df[df.Borough != "Not assigned"].reset_index(drop=True)
df1.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [83]:
df1['Neighbourhood'] =  [row[-2] if row[-1]=="Not assigned" else row[-1] for row in df1.itertuples()]
df1.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


If more than one neighborhood exists in one postal code area, those two rows will be combined into one row with the neighborhoods separated with a comma.

In [87]:
df1 = df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df1.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Using the .shape method to print the number of rows of the dataframe.

In [85]:
df1.shape

(103, 3)